In [2]:
import numpy as np
import cv2
import os

In [4]:
# import tensorflow as tf
import pandas as pd

In [5]:
from tensorflow import config

# Hide GPU from visible devices
config.set_visible_devices([], 'GPU')

## Video Preparation

Down sample number of images by a factor of 4. Save down 16 images from each, taking the images only during the start and end of the shot.

In [12]:
def get_frames_from_video(class_,video_path):
# frame shape : (480, 640, 3)
    print(f'Capturing frames from {video_path}')
    cap = cv2.VideoCapture(video_path)
    number_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'Video has number of frames: {number_frames}')
    if number_frames == 0:
        # Skip this file.
        return
    # we only want to save around a quarter of the video frames.
    output_frame = round(number_frames / 20) 
    current_frame = 0
    while(cap.isOpened() and current_frame != (number_frames)): 
        current_frame += 1
        # capture frame-by-frame
        ret, frame = cap.read()
        if ret: # check ! (some webcam's need a "warmup")
            file_name = os.path.split(video_path)[1]
            file_suffix = file_name.split('.')[0]
            name = os.path.join('/Users/adamwatson/Desktop/TENNIS/InputFrames/',class_,f'{file_suffix}_{current_frame}.jpg')
            # save every forth frame
            if current_frame % output_frame == 0:
#                 print(f'Saving frame to: {name}')
                cv2.imwrite(name, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # When everything is done release the capture
    output_frame = frame
    cap.release()
    cv2.destroyAllWindows()

Testing loading in a video.

In [13]:
cap = cv2.VideoCapture('/Users/adamwatson/Desktop/TENNIS/InputData/forehand/p12_fvolley_s1.avi')
number_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(number_frames)
ret, frame = cap.read()
#Display the resulting frame
cv2.imshow(my_video_name+' frame '+ str(frame_seq),gray)

#Set waitKey 
cv2.waitKey()
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

58


# Video to Frames

Convert tennis video dataset to frame images, reducing the number of frames by a factor of 4 to make processing more manageable.

In [25]:
# First, let's create the image folders ready for processing.
video_root = '/Users/adamwatson/Desktop/TENNIS/InputData/'
classes = ['backhand','backhand2hands','forehand','service','smash']
images_root = '/Users/adamwatson/Desktop/TENNIS/InputFrames/'

In [8]:
for class_ in classes:
    class_path = os.path.join(images_root,class_)
    print(class_path)
    if not os.path.exists(class_path):
        try:
            os.makedirs(class_path)
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [9]:
%%timeit
no_videos = 0
for class_ in classes:
    for root, dirs, files in os.walk(os.path.join(video_root,class_), topdown=False):
        for name in files:
            no_videos += 1
            get_frames_from_video(class_,os.path.join(root, name))
            print(f'Videos processed: {no_videos}')


In [10]:
# rename files into alphanumeric order
# for class_ in classes:
#     no_videos = 0
#     print(class_)
#     class_dir = os.path.join(images_root,class_)
#     print(class_dir)
#     for root, dirs, files in os.walk(class_dir):
#         # Sort list of files based on last modification time in ascending order
#         list_of_files = sorted(files)
#         for name in list_of_files:
#             no_videos += 1
#             rename_dir = os.path.join(class_dir,f'{class_}_image_{no_videos}.jpg')
#             print(rename_dir)
#             os.rename(os.path.join(class_dir,name), os.path.join(class_dir,f'{class_}_image_{no_videos}.jpg'))
#     print(f'Frames processed for class {class_}: {no_videos}')

In [11]:
# get_frames_from_video('forehand','/Users/adamwatson/Desktop/TENNIS/InputData/forehand/p12_fvolley_s1.avi')

## Load in training data.

Use the keras preprocessing function to load in test files from directory. Depending on the folder name, this will assign the training data a class based on it's root directory.

In [14]:
# from tensorflow.keras.preprocessing import image_dataset_from_directory

# img_width = 480
# img_height = 640
# batch_size = 32
# train_dir = '/Users/adamwatson/Desktop/TENNIS/InputFrames'
# train_set = image_dataset_from_directory(
#     train_dir,
#     label_mode='categorical',
#     batch_size=batch_size,
#     image_size=(img_width, img_height),
#     shuffle=True,
#     seed=123)
# #     validation_split=0.2,
# #     subset="training")

In [15]:
# print(train_set.class_names)
# print(type(train_set))
# print(train_set)

## Load in training data using ImageDataGenerator

Use the keras preprocessing function to load in test files from directory. Depending on the folder name, this will assign the training data a class based on it's root directory.

In [16]:
ROWS=480
COLS=600
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow import test

train_idg = ImageDataGenerator(
                               preprocessing_function=preprocess_input)
train_gen = train_idg.flow_from_directory(
    '/Users/adamwatson/Desktop/TENNIS/InputFrames/',
    target_size=(ROWS, COLS),
    batch_size = 16
)

Found 22983 images belonging to 2 classes.


In [17]:
train_gen

## Create new model

Pre-trained CNN network such as InceptionV3, extracts features from each video frame and passes them to a softmax layer that outputs probabilities corresponding to each class. The input video is classified by averaging all the probabilities of each frame in the video.

In [18]:
# restrict GPU usage to restrict OOM issues.
from tensorflow.config.experimental import list_physical_devices
gpus = list_physical_devices('gpu')
print(len(gpus))

0


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

# USE FLATTEN LAYER.

input_shape = (ROWS, COLS, 3)
nclass = len(train_gen.class_indices)

base_model = InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(ROWS, COLS,3))
# Freeze inception layers (don't train these layers).
base_model.trainable = False

In [22]:
add_model = Sequential()
add_model.add(base_model)
# add a dropout layer here
# add_model.add(Flatten())
# find out what global avg pooling 2D does.
# add a validation split.
add_model.add(Dropout(rate=0.3))
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(nclass, 
                    activation='softmax'))

In [23]:
model = add_model
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.SGD(learning_rate=0.01, 
                                       momentum=0.9),
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 13, 17, 2048)      21802784  
_________________________________________________________________
dropout (Dropout)            (None, 13, 17, 2048)      0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


## Train Model

In [24]:
history = model.fit(train_gen, epochs=5, batch_size = 128)

2021-11-28 19:40:44.367119: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-28 19:40:44.368521: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/5
 137/1437 [=>............................] - ETA: 43:53 - loss: 0.9360 - accuracy: 0.5693

KeyboardInterrupt: 

In [1]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined